In [1]:
!pip3 install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio==0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
     |██████████████▋                 | 834.1 MB 1.2 MB/s eta 0:14:03tcmalloc: large alloc 1147494400 bytes == 0x558f7c214000 @  0x7f1e84192615 0x558f4287e3bc 0x558f4295f18a 0x558f428811cd 0x558f42973b3d 0x558f428f5458 0x558f428f002f 0x558f42882aba 0x558f428f52c0 0x558f428f002f 0x558f42882aba 0x558f428f1cd4 0x558f42974986 0x558f428f1350 0x558f42974986 0x558f428f1350 0x558f42974986 0x558f428f1350 0x558f42882f19 0x558f428c6a79 0x558f42881b32 0x558f428f51dd 0x558f428f002f 0x558f42882aba 0x558f428f1cd4 0x558f428f002f 0x558f42882aba 0x558f428f0eae 0x558f428829da 0x558f428f1108 0x558f428f002f
     |██████████████████▌             | 1055.7 MB 1.2 MB/s eta 0:10:46tcmalloc: large alloc 1434370048 bytes == 0x558fc086a000 @  0x7f1e84192615 0x558f4287e3bc 0x558f4295f18a 0x558f428811cd 0x558f42973b3d 0x558f428f5458 0x558f428f002f 0x558f42882aba 0x558f428f52c0 0x558f428f002f 0x558f42882aba 0x558f428f1cd4 0x558f42974986 0x558f

In [2]:
!pip3 install transformers
!pip3 install hugsvision

     |████████████████████████████████| 3.5 MB 5.2 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 895 kB 73.7 MB/s 
     |████████████████████████████████| 6.8 MB 69.8 MB/s 
     |████████████████████████████████| 596 kB 74.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 431 kB 7.6 MB/s 
     |████████████████████████████████| 397 kB 78.7 MB/s 
     |████████████████████████████████| 527 kB 81.0 MB/s 
     |████████████████████████████████| 952 kB 70.6 MB/s 
     |████████████████████████████████| 133 kB 83.1 MB/s 
     |████████████████████████████████| 829 kB 70.6 MB/s 
     |████████████████████████████████| 1.1 MB 72.2 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
     |████████████████████████████████| 144 kB 77.7 MB/s 
     |████████████████████████████████| 271 k

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!cd XRay/
!ls -a
!rm -r /content/drive/MyDrive/medicalbob/COVID19Dataset/Xray/.ipynb_checkpoints

/bin/bash: line 0: cd: XRay/: No such file or directory
.  ..  .config	drive  sample_data
rm: cannot remove '/content/drive/MyDrive/medicalbob/COVID19Dataset/Xray/.ipynb_checkpoints': No such file or directory


In [9]:
import transformers
from transformers import AutoFeatureExtractor, DeiTForImageClassification ,DeiTFeatureExtractor , ViTForImageClassification , ViTFeatureExtractor , ViTModel
from PIL import Image
import requests
import torch
import argparse
from torchvision import datasets
from torchvision import transforms as tfc
from torchmetrics import Accuracy
from torch.utils.data import DataLoader
import tqdm
import os
import numpy as np
import math
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import requests
import hugsvision
print(hugsvision.__version__)
from hugsvision.inference.VisionClassifierInference import VisionClassifierInference
from hugsvision.dataio.VisionDataset import VisionDataset
from google.colab import drive

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

dataset_dir = "/content/drive/MyDrive/medicalbob/COVID19Dataset/Xray/"  #"/content/drive/MyDrive/medicalbob/COVID19Dataset/Xray/"   "/content/drive/MyDrive/medicalbob/COVID19Dataset/CT/" 
ratio = 0.2
no_cuda = False
# def train(num_epochs,no_cuda,save_dir, projectname, batch_size,dataset_dir,model_name, ratio = 0.2, lr = 2e-5):
    # device = torch.device('cuda' if torch.cuda.is_available() and not no_cuda else 'cpu')



train_ds, test_ds, id2label, label2id = VisionDataset.fromImageFolder( 
    dataset_dir 
    ,test_ratio=ratio
    ,balanced=True
    ,augmentation=False
    ,torch_vision=True
)



0.75.3


FileNotFoundError: ignored

In [ ]:
from hugsvision.nnet.VisionClassifierTrainer import VisionClassifierTrainer

lr, num_epochs = 2e-5, 20
save_dir = "/content/drive/MyDrive/medicalbob/checkpoint/"
projectname = "XrayDeitExp"
model_pretrained = "google/vit-base-patch16-224-in21k"
model_name = "Deit"
batch_size = 128
if model_name == 'Deit':
    model_pretrained = "facebook/deit-base-distilled-patch16-224"
    trainer = VisionClassifierTrainer(
        model_name=projectname,
        train=train_ds,
        test=test_ds,
        output_dir=save_dir,
        max_epochs=num_epochs,
        batch_size=batch_size,  # On RTX 2080 Ti
        lr	= lr,
        # fp16	     = False,
        model = DeiTForImageClassification.from_pretrained(
            model_pretrained,
            num_labels = len(label2id),
            label2id   = label2id,
            id2label   = id2label
        ),
        feature_extractor = DeiTFeatureExtractor.from_pretrained(
            model_pretrained,
        ),
    )
else:
    trainer = VisionClassifierTrainer(
        model_name=projectname,
        train=train_ds,
        test=test_ds,
        output_dir=save_dir,
        max_epochs=num_epochs,
        batch_size=batch_size,  # On RTX 2080 Ti
        lr	= lr,
        # fp16	     = False,
        model = ViTForImageClassification.from_pretrained(
            model_pretrained,
            num_labels = len(label2id),
            label2id   = label2id,
            id2label   = id2label
        ),
        feature_extractor = ViTFeatureExtractor.from_pretrained(
            model_pretrained,
        ),
    )

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.7/dist-pa

{'0': 'COVID', '1': 'NORMAL'}
{'COVID': '0', 'NORMAL': '1'}
Trainer builded!
Start Training!


Epoch,Training Loss,Validation Loss
1,No log,0.285310
2,No log,0.153234
3,No log,0.117769
4,No log,0.100534
5,No log,0.098611
6,No log,0.095271
7,No log,0.087546
8,No log,0.084675
9,No log,0.112339
10,No log,0.114334


***** Running Evaluation *****
  Num examples = 1052
  Batch size = 128
***** Running Evaluation *****
  Num examples = 1052
  Batch size = 128
***** Running Evaluation *****
  Num examples = 1052
  Batch size = 128
***** Running Evaluation *****
  Num examples = 1052
  Batch size = 128
***** Running Evaluation *****
  Num examples = 1052
  Batch size = 128
***** Running Evaluation *****
  Num examples = 1052
  Batch size = 128
***** Running Evaluation *****
  Num examples = 1052
  Batch size = 128
***** Running Evaluation *****
  Num examples = 1052
  Batch size = 128
***** Running Evaluation *****
  Num examples = 1052
  Batch size = 128
***** Running Evaluation *****
  Num examples = 1052
  Batch size = 128
***** Running Evaluation *****
  Num examples = 1052
  Batch size = 128
***** Running Evaluation *****
  Num examples = 1052
  Batch size = 128
***** Running Evaluation *****
  Num examples = 1052
  Batch size = 128
***** Running Evaluation *****
  Num examples = 1052
  Batch siz

Model saved at: /content/drive/MyDrive/medicalbob/checkpoint/CTVITEXP/20_2022-02-17-09-22-06
